In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_openml
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris
from sklearn.preprocessing import LabelEncoder

In [46]:
df = pd.read_csv('/content/Titanic-Dataset.csv')

In [47]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [50]:
df.isnull().sum()

,0
PassengerId,0
Survived,0
Pclass,0
Name,0
Sex,0
Age,177
SibSp,0
Parch,0
Ticket,0
Fare,0


In [52]:
df.fillna({'Age': np.median(df['Age'])},inplace=True)

In [53]:
columns=['Name','Ticket','PassengerId','Cabin']
df=df.drop(columns,axis=1)

In [64]:
df.dropna(inplace=True)

In [65]:
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


In [66]:
x=df.iloc[:,1:]
y=df.iloc[:,0]

In [67]:
columns=['Sex','Embarked']

for features in columns:
  x[features]=LabelEncoder().fit_transform(x[features])

In [68]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

In [69]:
x = scaler.fit_transform(x)

In [70]:
X_train,X_test,Y_train,Y_test=train_test_split(x,y,test_size=0.2,random_state=40, stratify=y)

##Logistic Regression from scratch

In [86]:
class Logistic_regression:
    def __init__(self):
        self.parameters = {}

    def forward_propagation(self, X_train):
        predictions = np.dot(X_train, self.parameters['w']) + self.parameters['b']
        return predictions

    def sigmoid(self, predictions):
        predictions = np.clip(predictions, -500, 500)
        return 1 / (1 + np.exp(-predictions))

    def compute_cost(self, A, y_train, epsilon = 10e-6):
        m = y_train.shape[0]
        y_train = y_train.astype(np.float64)
        A = A.astype(np.float64).flatten()
        cost = -(1 / m) * np.sum(y_train * np.log(A + epsilon) + (1 - y_train) * np.log(1 - A + epsilon))
        return cost

    def backward_propagation(self, X_train, A, y_train):
        m = y_train.shape[0]
        if A.ndim > 1:
            A = A.flatten()
        dw = (1 / m) * np.dot(X_train.T, (A - y_train))
        db = (1 / m) * np.sum(A - y_train)
        return {"dw": dw, "db": db}

    def update_parameters(self, derivates, lr):
        self.parameters['w'] = self.parameters['w'] - lr * derivates['dw']
        self.parameters['b'] = self.parameters['b'] - lr * derivates['db']

    def initialize_parameters(self, X_train):
        # np.random.seed(42)
        self.parameters['w'] = np.random.randn(X_train.shape[1]) * 0.01
        self.parameters['b'] = 0


    def fit(self, X_train, y_train, num_iterations=2000, lr=10e-3):
        self.initialize_parameters(X_train)
        for i in range(num_iterations):
            predictions = self.forward_propagation(X_train)
            A = self.sigmoid(predictions)
            cost = self.compute_cost(A, y_train)
            derivates = self.backward_propagation(X_train, A, y_train)
            self.update_parameters(derivates, lr)
            if i % 100 == 99:
                print(f"Iteration {i+1}: Cost = {cost}")

    def predict(self, X_test):
        A = self.sigmoid(self.forward_propagation(X_test))
        return (A > 0.5).astype(int)

find error cost =-1/m * sum (ylogp(p(x)) + (1-y)log(1-p(x))

In [87]:
model1=Logistic_regression()
model1.fit(X_train,Y_train)

Iteration 100: Cost = 0.5973809511092598
Iteration 200: Cost = 0.546836618328263
Iteration 300: Cost = 0.5176732099996734
Iteration 400: Cost = 0.49934107095335006
Iteration 500: Cost = 0.4870830042334614
Iteration 600: Cost = 0.4785023526205007
Iteration 700: Cost = 0.47228131972647336
Iteration 800: Cost = 0.46764409330391205
Iteration 900: Cost = 0.464108616273429
Iteration 1000: Cost = 0.4613621076178831
Iteration 1100: Cost = 0.459194328444724
Iteration 1200: Cost = 0.4574597688887819
Iteration 1300: Cost = 0.45605520690637996
Iteration 1400: Cost = 0.45490585964114943
Iteration 1500: Cost = 0.4539565489582762
Iteration 1600: Cost = 0.4531659031436865
Iteration 1700: Cost = 0.45250245670993683
Iteration 1800: Cost = 0.45194197021232646
Iteration 1900: Cost = 0.4514655534620301
Iteration 2000: Cost = 0.45105832917759


In [88]:
accuracy = np.mean(model1.predict(X_test) == Y_test)
print(f"Accuracy:{accuracy}")

Accuracy:0.7832167832167832


##Model

In [74]:
model=LogisticRegression()
model.fit(X_train,Y_train)

LogisticRegression()

In [76]:
from sklearn.metrics import accuracy_score
y_pred=model.predict(X_test)
accuracy=accuracy_score(Y_test,y_pred)
print(f"Accuracy:{accuracy}")

Accuracy:0.7972027972027972
